In [1]:
from Q12v2 import *
import numpy as np
import pandas as pd

In [2]:
tokenized = open("Tokenized-2Q1.txt", "r", encoding="utf-8")
Para = tokenized.read()
Para = Para.split("\n")
remove = ['‘', '?', '.', '’', ',']

for sentence in range(len(Para)):
    Para[sentence] = Para[sentence].split(" ")

    for word in range(len(Para[sentence])):
        if "\u200c" in Para[sentence][word]:
            Para[sentence][word] = " ".join(Para[sentence][word].split("\u200c"))

        clean = ""
        for ch in Para[sentence][word]:
            if ch not in remove:
                clean += ch
        Para[sentence][word] = clean.lower()

In [3]:
Train, Validate, Test = Divider(Para, Validation = 1000, Test = 10)

In [4]:
len(Train), len(Validate), len(Test), len(Para)

(427548, 1000, 10, 428558)

In [6]:
Conut_List, Probability_List = NGram(4, Train, Smoothing = None)


In [39]:
import numpy as np

def Deleted_Interpolated_Smoothing(Gram=4, L1=None, L2=None, L3=None, L4=None,
                                   G1=None, G2=None, G3=None, G4=None, Data=None):
    if Gram < 4:
        return
    i = 0
    l1, l2, l3, l4 = 1,1,1,1
    Quadrigrams = []
    PQ = {}

    for sentence in Data:
        for i in range(len(sentence)-3):
            Quadrigrams.append(sentence[i: i + 4])

    best_lambdas = None
    best_log_likelihood = -float('inf')

    # Lambda loops
    for lambda4 in np.arange(0.0, 1.1, 0.1):
        for lambda3 in np.arange(0.0, 1.1 - lambda4, 0.1):
            for lambda2 in np.arange(0.0, 1.1 - lambda4 - lambda3, 0.1):
                lambda1 = 1.0 - lambda4 - lambda3 - lambda2
                log_likelihood = 0

                for Q in Quadrigrams:
                    # Get probabilities (handle dict returned by Good_Turing_Smoothing)
                    P1 = G1.get(Q[3], Good_Turing_Smoothing(Gram=1, L1=L1, G1=G1, Data=[[Q[3]]]))
                    if isinstance(P1, dict):
                        P1 = list(P1.values())[0]

                    P2 = G2.get(tuple(Q[2:4]), Good_Turing_Smoothing(Gram=2, L1=L1, L2=L2, G2=G2, Data=[Q[2:4]]))
                    if isinstance(P2, dict):
                        P2 = list(P2.values())[0]

                    P3 = G3.get(tuple(Q[1:4]), Good_Turing_Smoothing(Gram=3, L1=L1, L2=L2, L3=L3, G3=G3, Data=[Q[1:4]]))
                    if isinstance(P3, dict):
                        P3 = list(P3.values())[0]

                    P4 = G4.get(tuple(Q), Good_Turing_Smoothing(Gram=4, L1=L1, L2=L2, L3=L3, L4=L4,
                                                                 G1=G1, G2=G2, G3=G3, G4=G4, Data=[Q]))
                    if isinstance(P4, dict):
                        P4 = list(P4.values())[0]

                    # Interpolated probability
                    P_interp = lambda4*P4 + lambda3*P3 + lambda2*P2 + lambda1*P1
                    PQ[tuple(Q)] = P_interp

                    log_likelihood += np.log(P_interp)

                if log_likelihood > best_log_likelihood:
                    best_log_likelihood = log_likelihood
                    best_lambdas = (lambda1, lambda2, lambda3, lambda4)

    return best_lambdas, PQ


In [ ]:
Bl = Deleted_Interpolated_Smoothing(
    Gram=4,
    L1=Conut_List[0],
    L2=Conut_List[1],
    L3=Conut_List[2],
    L4=Conut_List[3],
    G1=Probability_List[0],
    G2=Probability_List[1],
    G3=Probability_List[2],
    G4=Probability_List[3],
    Data=Test[:10]
)

[['అన్నమాట']]
[['వ్యూస్', 'అన్నమాట']]
[['కోట్ల', 'వ్యూస్', 'అన్నమాట']]
[['3', 'కోట్ల', 'వ్యూస్', 'అన్నమాట']]
[['నెటిజన్లు']]
[['చూసిన', 'నెటిజన్లు']]
[['వీడియోను', 'చూసిన', 'నెటిజన్లు']]
[['ఆ', 'వీడియోను', 'చూసిన', 'నెటిజన్లు']]
[['అయితే']]
[['నెటిజన్లు', 'అయితే']]
[['చూసిన', 'నెటిజన్లు', 'అయితే']]
[['వీడియోను', 'చూసిన', 'నెటిజన్లు', 'అయితే']]
[['స్టంట్కు']]
[['చేసిన', 'స్టంట్కు']]
[['యువతి', 'చేసిన', 'స్టంట్కు']]
[['ఆ', 'యువతి', 'చేసిన', 'స్టంట్కు']]
[['ఫిదా']]
[['స్టంట్కు', 'ఫిదా']]
[['చేసిన', 'స్టంట్కు', 'ఫిదా']]
[['యువతి', 'చేసిన', 'స్టంట్కు', 'ఫిదా']]
[['అయిపోతున్నారు']]
[['ఫిదా', 'అయిపోతున్నారు']]
[['స్టంట్కు', 'ఫిదా', 'అయిపోతున్నారు']]
[['చేసిన', 'స్టంట్కు', 'ఫిదా', 'అయిపోతున్నారు']]
[['యువతి']]
[['ఆ', 'యువతి']]
[['కూడా', 'ఆ', 'యువతి']]
[['మీరు', 'కూడా', 'ఆ', 'యువతి']]
[['చేసిన']]
[['యువతి', 'చేసిన']]
[['ఆ', 'యువతి', 'చేసిన']]
[['కూడా', 'ఆ', 'యువతి', 'చేసిన']]
[['స్టంట్']]
[['చేసిన', 'స్టంట్']]
[['యువతి', 'చేసిన', 'స్టంట్']]
[['ఆ', 'యువతి', 'చేసిన', 'స్టంట్']]
[['వీడియోను']]
[['

C:\Users\jodya\AppData\Local\Temp\ipykernel_9152\1327594170.py:49: RuntimeWarning: invalid value encountered in log
  log_likelihood += np.log(P_interp)


[['ఆ', 'వీడియోను', 'చూసిన', 'నెటిజన్లు']]
[['అయితే']]
[['నెటిజన్లు', 'అయితే']]
[['చూసిన', 'నెటిజన్లు', 'అయితే']]
[['వీడియోను', 'చూసిన', 'నెటిజన్లు', 'అయితే']]
[['స్టంట్కు']]
[['చేసిన', 'స్టంట్కు']]
[['యువతి', 'చేసిన', 'స్టంట్కు']]
[['ఆ', 'యువతి', 'చేసిన', 'స్టంట్కు']]
[['ఫిదా']]
[['స్టంట్కు', 'ఫిదా']]
[['చేసిన', 'స్టంట్కు', 'ఫిదా']]
[['యువతి', 'చేసిన', 'స్టంట్కు', 'ఫిదా']]
[['అయిపోతున్నారు']]
[['ఫిదా', 'అయిపోతున్నారు']]
[['స్టంట్కు', 'ఫిదా', 'అయిపోతున్నారు']]
[['చేసిన', 'స్టంట్కు', 'ఫిదా', 'అయిపోతున్నారు']]
[['యువతి']]
[['ఆ', 'యువతి']]
[['కూడా', 'ఆ', 'యువతి']]
[['మీరు', 'కూడా', 'ఆ', 'యువతి']]
[['చేసిన']]
[['యువతి', 'చేసిన']]
[['ఆ', 'యువతి', 'చేసిన']]
[['కూడా', 'ఆ', 'యువతి', 'చేసిన']]
[['స్టంట్']]
[['చేసిన', 'స్టంట్']]
[['యువతి', 'చేసిన', 'స్టంట్']]
[['ఆ', 'యువతి', 'చేసిన', 'స్టంట్']]
[['వీడియోను']]
[['స్టంట్', 'వీడియోను']]
[['చేసిన', 'స్టంట్', 'వీడియోను']]
[['యువతి', 'చేసిన', 'స్టంట్', 'వీడియోను']]
[['చూసేయండి']]
[['వీడియోను', 'చూసేయండి']]
[['స్టంట్', 'వీడియోను', 'చూసేయండి']]
[['చేసిన'